In [1]:
import pandas as pd
import gc
from sklearn import preprocessing
import time
import math
import pickle
import numpy as np

In [2]:
total =  pickle.load(open('../../data/raw/total_high_level.pkl','rb+'))
total = total.reset_index(drop=True)

In [3]:
#def timestamp_datetime(value):
#    format = '%Y-%m-%d %H:%M:%S'
#    value = time.localtime(value)
#    dt = time.strftime(format, value)
#    return dt
#total['context_timestamp'] = total['context_timestamp'].apply(timestamp_datetime)
#total['context_timestamp'] = pd.to_datetime(total['context_timestamp'])
#total["day"] = total["context_timestamp"].dt.day
#total["hour"] = total["context_timestamp"].dt.hour

In [7]:
data = total.loc[(total["context_timestamp"] >= "20180907"),
               ["hour","day","context_timestamp","context_page_id","predict_category_property","item_property_list","item_category_list","is_trade"]]

In [8]:
#data = pd.get_dummies(data, columns=['hour'])
data = pd.get_dummies(data, columns=['day'])

In [9]:
#查询与实际匹配情况
#询词预测的类目
def get_match_num(x):
    num = 0.0
    lists = x.split(";")
    for category in lists:
        category = category.split(":")
        if len(category) >=2 and category[1].split(",")[0] != "-1":
            num+=1.0
    return num

#根据查询词预测的属性匹配数
def get_match_property_num(x):
    predicts = set()
    lists = x["predict_category_property"].split(";")
    for category in lists:
        category = category.split(":")
        if len(category) >=2:
            proerties = category[1].split(",")
            if proerties[0] != "-1":
                for proerty in proerties:
                    predicts.add(proerty)
    true = set()
    lists = x["item_property_list"].split(";")
    for proerty in lists:
        true.add(proerty)
    return len(true & predicts)*1.0

# 查询词汇预测类目与该商品匹配的个数
def get_match_item_category(x):
    query_category_set = set()
    lists = x["predict_category_property"].split(";")
    for category in lists:
        category = category.split(":")[0]
        if category != "-1":
            query_category_set.add(category)
    item_category_set = set(x["item_category_list"].split(";"))
    return len(item_category_set & query_category_set)*1.0


# 查询词汇
data["property_num"] = data["item_property_list"].apply(lambda x : len(x.split(";"))*1.0)
print("计算：property_num完毕")
data["category_num"] = data["item_category_list"].apply(lambda x : len(x.split(";"))*1.0)
print("计算：category_num完毕")
#查询词汇中类目总数
data["predict_category_num"] = data["predict_category_property"].apply(lambda x: len(x.split(";"))*1.0)
print("计算：predict_category_num完毕")

#查询词汇中，预测匹配类目的总数,即不是-1的总类目数
data["match_category_num"] = data["predict_category_property"].apply(lambda x: get_match_num(x))
# 预测匹配类目/查询词汇类目总数
data["match_category_ratio"] = data.apply(lambda x: x["match_category_num"]/x["predict_category_num"], axis=1)
# 一共匹配属性的个数
data["match_proetry_num"] = data.apply(lambda x: get_match_property_num(x), axis=1)
# 匹配属性个数/该商品属性总数
data["match_proetry_num_ratio"] = data.apply(lambda x: x["match_proetry_num"]/x["property_num"], axis=1)
# 查询词汇与该商品类目匹配的个数
data["match_item_category_num"] = data.apply(lambda x: get_match_item_category(x), axis=1)
# 查询词汇与该商品类目匹配的个数/该商品类目总数
data["predict_category_item_category_ratio"] = data.apply(lambda x : x["match_item_category_num"]/x["category_num"], axis=1)


计算：property_num完毕
计算：category_num完毕
计算：predict_category_num完毕


In [10]:
data.pop("item_category_list")
data.pop("item_property_list")
data.pop("predict_category_property")
context_timestamp = data.pop("context_timestamp")

In [11]:
data.to_pickle('../../data/processed/context_feature.pkl')

In [12]:
context_timestamp.min()

Timestamp('2018-09-07 00:00:00')

In [13]:
data.shape

(1597063, 13)